In [1]:
import requests
import bs4
import pandas as pd
from urllib.parse import urlparse, urljoin
import ngram_value
import size

In [2]:
url = input('Enter the url :')

Enter the url :https://www.udacity.com/


In [3]:
# adds (http://)  if not included
if 'http://' in url or 'https://' in url:
    pass
else:
    url = 'http://' + url    

In [4]:
url

'https://www.udacity.com/'

In [5]:
domain_name = urlparse(url).netloc   # returns the domain name of the website

In [6]:
#checks whether link is valid or not
def check_url(url):
    try:
        response = requests.get(url)
        #print("URL is valid and exists on the Internet")
        return True
    except:
        print("URL does not exist on the Internet")
        return False

In [7]:
check_url(url)

True

In [9]:
# returns the homepage of the web_link
def home_page(url):
    count = 0
    url = url + '/'
    url_len = len(url)
    for i in range(url_len):
        if '/' == url[i]:
            count +=1
            if count == 3:
                break
    return i    

In [10]:
val = home_page(url)
url = url[0:val]
print(url)

https://www.udacity.com


In [11]:
#finding all the links 

valid_link = set()  # set variable to remove duplicate values

def link_gen(url):
    
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.text,'lxml')
    
    for link in soup.find_all('a',href=True):                            # loop through all links
        
        first_char = link['href'][0]
        
        if (check_url(link['href']) and  '#' not in link['href'][0]):    # excluding garbage link
            if first_char != '/':
                valid_link.add(link['href'])
                #print(link['href'])
        
        if first_char == '/':                                            # adding url to few internal link                                      
            final_link = url + link['href']
            valid_link.add(final_link)
            #print(final_link)
                

In [12]:
link_gen(url)

URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet
URL does not exist on the Internet


In [13]:
valid_num = len(valid_link)  # no of valid links

In [14]:
# necessary list variables to store data in csv

internal_links = []
external_links = []

title_tag = []
meta_value = []
internal_value = []
external_value = []
uni_value = []
bi_value = []
size_value = []




def get_title(valid_link):
    
    for link in valid_link:                                 # looping through all valid link
        
        try:
            res = requests.get(link)                        # try except block if it throws error
            soup = bs4.BeautifulSoup(res.text,'lxml')
        except:
            continue
        
        
        if len(soup.select('title')) != 0:                 # selecting title of the webpage and storing it in list
            title_tag.append(soup.select('title')[0].text)
        else:
            title_tag.append('N/A')
        
        
        description = soup.find('meta', attrs={'name':'og:description'}) or soup.find('meta', attrs={'property':'description'}) or soup.find('meta', attrs={'name':'description'})
        if description:                                   # adding desc from meta tag to list
           description_text = description.get('content')
        else:
            description_text = 'N/A'
            
        meta_value.append(description_text)
        
        
        
        if domain_name in link:                           # classfying the internal and external links                      
            internal_links.append(link)
            internal_value.append('yes')
            external_value.append('no')
        else:
            external_links.append(link)
            internal_value.append('no')
            external_value.append('yes')
        
        a,b = ngram_value.uni_bi_values(link)            # appending unigrams,bigrams
        uni_value.append(a)
        bi_value.append(b)
        
        
        temp = size.total_size(link)                    # adding size of the web page to list
        size_value.append(temp)
            
        
            
        
        
        
        
        
    

In [15]:
get_title(valid_link)

https://www.udacity.com/course/ux-designer-nanodegree--nd578
Denied
https://www.udacity.com/school-of-ai
Denied
https://www.udacity.com/course/data-visualization-nanodegree--nd197
Denied
https://www.udacity.com/scholarships
Denied
https://www.instagram.com/udacity/
https://www.udacity.com/course/ai-for-healthcare-nanodegree--nd320
Denied
https://www.udacity.com/course/aws-cloud-architect-nanodegree--nd063
Denied
https://www.udacity.com/course/intro-to-programming-nanodegree--nd000
Denied
https://www.udacity.com/school-of-autonomous-systems
Denied
https://www.udacity.com/course/data-analyst-nanodegree--nd002
Denied
https://www.udacity.com/course/intel-edge-ai-for-iot-developers-nanodegree--nd131
Denied
https://www.udacity.com/course/deep-learning-nanodegree--nd101
Denied
https://www.udacity.com/course/rpa-developer-nanodegree--nd340
Denied
https://www.udacity.com/course/data-streaming-nanodegree--nd029
Denied
https://www.udacity.com/course/ai-product-manager-nanodegree--nd088
Denied
htt

In [16]:
new_valid_link = list(valid_link)   # set to list

In [17]:
final_result = {}

for i in range(valid_num):         # looping through each valid link and storing coresponding values
    
    final_result[i] = [new_valid_link[i],title_tag[i],meta_value[i],internal_value[i],external_value[i],uni_value[i],bi_value[i],size_value[i]] 

In [18]:
final_result_df = pd.DataFrame.from_dict(final_result,orient='index', columns=['Links','Title','Meta-data(DESC)','Internal','External','unigram','bi-gram','web-page size'])

In [19]:
final_result_df.tail()    #printing Df

,Links,Title,Meta-data(DESC),Internal,External,unigram,bi-gram,web-page size
85,https://www.udacity.com/online-learning-hub/wfh,Online Learning | Udacity,Many people are experiencing online learning f...,yes,no,"[Â, program, Nanodegree, Udacity, Data, I, Lea...","[(MoreFree, ProgramFreeSchool), (ProgramFreeSc...",N/A
86,https://www.udacity.com/jobs,Jobs at Udacity,Join Udacity in our mission to democratize edu...,yes,no,"[Â, program, Nanodegree, Udacity, Data, I, Lea...","[(MoreFree, ProgramFreeSchool), (ProgramFreeSc...",N/A
87,https://www.udacity.com/course/intro-to-machin...,Intro to Machine Learning with Pytorch Course,Learn foundational machine learning algorithms...,yes,no,"[Â, program, Nanodegree, Udacity, Data, I, Lea...","[(MoreFree, ProgramFreeSchool), (ProgramFreeSc...",N/A
88,https://www.udacity.com/us,About | Udacity,Udacity is where lifelong learners come to lea...,yes,no,"[Â, program, Nanodegree, Udacity, Data, I, Lea...","[(MoreFree, ProgramFreeSchool), (ProgramFreeSc...",N/A
89,https://www.udacity.com/course/android-kotlin-...,Android Kotlin Developer Online Course,Start your career as an Android Kotlin develop...,yes,no,"[Â, program, Nanodegree, Udacity, Data, I, Lea...","[(MoreFree, ProgramFreeSchool), (ProgramFreeSc...",N/A


In [20]:
final_result_df.to_csv('udacity.csv')    #downloading csv file

In [26]:
print('No of Top level pages: ' + str(valid_num))  # total top level pages

No of Top level pages: 90


In [32]:
print('No of Internal links: ' + str(len(internal_links)))

No of Internal links: 79


In [31]:
print('No of Internal links: ' + str(len(external_links)))

No of Internal links:11
